# [NTDS'18] milestone 2: network models
[ntds'18]: https://github.com/mdeff/ntds_2018

[Hermina Petric Maretic](https://people.epfl.ch/hermina.petricmaretic), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: 31
* Students: Dilara Günay, Derin Sinan Bursa, Othman Benchekroun, Sinan Gökçe
* Dataset: IMDb Films and Crew

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to two sentences.
* Code has to be clean.
* In the first part, you cannot import any other library than we imported. In the second part, you are allowed to import any library you want.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective

The purpose of this milestone is to explore various random network models, analyse their properties and compare them to your network. In the first part of the milestone you will implement two random graph models and try to fit them to your network. In this part you are not allowed to use any additional package. In the second part of the milestone you will choose a third random graph model that you think shares some properties with your network. You will be allowed to use additional packages to construct this network, but you must explain your network choice. Finally, make your code as clean as possible, and keep your textual answers short.

## Part 0

Import the adjacency matrix of your graph that you constructed in milestone 1, as well as the number of nodes and edges of your network.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# getting adjacency and edges matrices
adjacency = pd.read_csv('data/adjacency.csv')
edges =  pd.read_csv('data/edges.csv')

In [ ]:
edges.head()

In [ ]:
adjacency.drop('Unnamed: 0', axis = 1, inplace = True)
adjacency = adjacency.values
np.set_printoptions(suppress = True)
# calculating the number of nodes and edges of our graph
n_nodes =  len(adjacency)
n_edges = len(edges)

In [ ]:
n_edges

## Part 1

**For the computation of this part of the milestone you are only allowed to use the packages that have been imported in the cell below.**

In [ ]:
%matplotlib inline

import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

In [ ]:
assda = np.zeros(shape = (5,5))
for i in range(5):
    for j in range(i+1,5):
        assda[i,j] = 0.1*j
assda        

In [ ]:
assda = np.zeros(shape = (3,3))
il1 = np.tril_indices(3,-1)
iu1 = np.triu_indices(3,1)
assda[0,1] = 1
assda[0,2] = 2
assda[1,2] = 3
assda[il1] = assda[iu1]
assda

In [ ]:
assda = np.zeros(shape = (3,3))
assda[0,1] = 1
assda[0,2] = 2
assda[1,2] = 3
assda
A = assda + np.transpose(assda)
A

### Question 1

Create a function that constructs an Erdős–Rényi graph.

In [ ]:
def erdos_renyi(n, p, seed=None):
    
    """Create an instance from the Erdos-Renyi graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    p: float
        Edge probability. A number between 0 and 1.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """
    
    adjacency = np.zeros(shape=(n,n))
    index_lower = np.tril_indices(n,-1)
    index_upper = np.triu_indices(n,1)
    
    for i in range(n):
        for j in range(i+1,n):
            random.seed(seed)
            r = random.random()               
            if r <= p:
                adjacency[i,j] = 1
            else:
                continue
                
    adjacency[index_lower] = adjacency[index_upper]
    
    return adjacency

In [ ]:
er = erdos_renyi(5, 0.6, 9765) #seed = 9765
plt.spy(er)
plt.title('Erdos-Renyi (5, 0.6)')

In [ ]:
er = erdos_renyi(10, 0.4, 7468) #seed = 7468
plt.spy(er)
plt.title('Erdos-Renyi (10, 0.4)')

### Question 2

Use the function to create a random Erdos-Renyi graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
prob = 2*n_edges/(n_nodes*(n_nodes-1)) #Random Networks slide 9/52
print(prob)

In [ ]:
random_er = erdos_renyi(n=n_nodes, p=prob)
plt.spy(random_er)
plt.title('Erdos-Renyi (10, {:.4f})'.format(prob))

In [ ]:
edges_erdos = np.count_nonzero(np.triu(random_er))
edges_erdos

**Your answer here.**

2835900 with prob 0.4

206969 with prop calculated (206391) 

### Question 3

Create a function that constructs a Barabási-Albert graph.

**Explanation:** The following function creates the Barabasi-Albert graph. It takes the number of nodes, the number of edges to attach from a new node to existing nodes and number of nodes for the initial connected network that is equal to 2 by default. Also, optionally a seed for the random number generator can be given as an input to get reproducible results. In the function itself, firstly an initial connected network is formed with m0 nodes. Later, one more new node is connected to m nodes until there are n number of nodes (given as input) in the connected graph.

In [ ]:
def barabasi_albert(n, m, m0=2, seed=None):
    """Create an instance from the Barabasi-Albert graph model.
    
    Parameters
    ----------
    n: int
        Size of the graph.
    m: int
        Number of edges to attach from a new node to existing nodes.
    m0: int (optional)
        Number of nodes for the inital connected network.
    seed: int (optional)
        Seed for the random number generator. To get reproducible results.
    
    Returns
    -------
    adjacency
        The adjacency matrix of a graph.
    """    
    random.seed(seed)
    # Add m0 initial nodes
    adjacency = np.zeros(shape=(n,n))
    for k in range(m0):
        for l in range(k+1,m0):
            adjacency[k,l] = 1
            
    # List of existing nodes, with nodes repeated once for each adjacent edge 
    visited_nodes = list(range(m0))
    
    # Target nodes for new edges 
    target = random.sample(range(len(visited_nodes)),m)
    
    # Start adding the other n-m nodes. The first node is m. 
    for i in range(m0, n):
        for j in range(m):
            adjacency[target[j],i] = 1
        visited_nodes.extend([i])
        target = random.sample(range(len(visited_nodes)),m)
    
    adjacency = adjacency + np.transpose(adjacency)
    
    
    return adjacency

In [ ]:
ba = barabasi_albert(5, 1, 2, 9087)
plt.spy(ba)
plt.title('Barabasi-Albert (5, 1)')

In [ ]:
ba = barabasi_albert(10, 2, 3, 8708)
plt.spy(ba)
plt.title('Barabasi-Albert (10, 2)')

### Question 4

Use the function to create a random Barabási-Albert graph. Choose the parameters such that number of nodes is the same as in your graph, and the number of edges similar. You don't need to set the random seed. Comment on your choice of parameters.

In [ ]:
n = n_nodes
m = int(n_edges/n_nodes)
m0 = int(n_edges/n_nodes)

In [ ]:
random_ba = barabasi_albert(n, m, m0)
plt.spy(random_ba)
plt.title('Barabasi-Albert (10, {})'.format(m))

In [ ]:
edges_barabasi = np.count_nonzero(np.triu(random_ba))
edges_barabasi

**Your answer here**

Firstly, we chose the n as the original number of nodes of our graph. In a Barabasi-Albert graph, number of edges is equal to m multiplied with number of nodes where m is the number of edges per node. Hence we have chosen the input m as number of edges/number of nodes. Additionally, the number of nodes for the initial graph that is m0 is also equal to number of edges/number of nodes. In the end, we have obtained 201879 edges where our original number of edges is 206391. Hence, it could be stated that we have obtained a similar number of edges with these parameters.

### Question 5

Compare the number of edges in all three networks (your real network, the Erdős–Rényi network, and the Barabási-Albert netowk).

In [ ]:
print("Number of edges in our network : ", n_edges)
print("Number of edges in the Erdős–Rényi network : ", edges_erdos)
print("Number of edges in the Barabási-Albert network : ", edges_barabasi)

### Question 6

Implement a function that computes the [Kullback–Leibler (KL) divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) between two probability distributions.
We'll use it to compare the degree distributions of networks.

In [ ]:
def kl_divergence(p, q):
    """Compute the KL divergence between probability distributions of degrees of two networks.
    
    Parameters
    ----------
    p: np.array
        Probability distribution of degrees of the 1st graph.
    q: np.array
        Probability distribution of degrees of the 2nd graph.
    
    Returns
    -------
    kl
        The KL divergence between tmhe two distributions.
    """
    
    assert len(p) == len(q)
    
    kl = 0
    
    for i in range(len(p)):
        kl = kl + (p[i] * np.log(p[i]/q[i]))
    
    return kl

In [ ]:
p_test = np.array([0.2, 0.2, 0.2, 0.4])
q_test = np.array([0.3, 0.3, 0.1, 0.3])
kl_divergence(p_test, q_test)

### Question 7: 

Compare the degree distribution of your network to each of the two synthetic ones, in terms of KL divergence.

**Hint:** Make sure you normalise your degree distributions to make them valid probability distributions.

**Hint:** Make sure none of the graphs have disconnected nodes, as KL divergence will not be defined in that case. If that happens with one of the randomly generated networks, you can regenerate it and keep the seed that gives you no disconnected nodes.

#### Our network :

In [ ]:
type(adjacency)

In [ ]:
degree_adj = np.count_nonzero(adjacency, axis=1)
unique_adj, counts_adj = np.unique(degree_adj, return_counts=True)
counts_adj = counts_adj / float(n_nodes)
weights_adj = dict(zip(unique_adj, counts_adj))

In [ ]:
values_adj = set(range(1, 624))
degrees_adj = set(degree_adj)
print(sorted(values_adj - degrees_adj))

#### Erdős–Rényi network : 

In [ ]:
type(random_er)

In [ ]:
degree_er = np.count_nonzero(random_er, axis=1)
unique_er, counts_er = np.unique(degree_er, return_counts=True)
counts_er = counts_er / float(n_nodes)
weights_er = dict(zip(unique_er, counts_er))

In [ ]:
values_er = set(range(74, 146))
degrees_er = set(degree_er)
print(values_er - degrees_er)

In [ ]:
bins_er = [1]+[75+2*i for i in range(36)]+[623]

In [ ]:
dist_er = np.histogram(unique_er, weights=counts_er, bins = bins_er)[0]
dist_adj_er = np.histogram(unique_adj, weights=counts_adj, bins = bins_er)[0]

In [ ]:
kl_adj_er = kl_divergence(dist_er, dist_adj_er)
print(kl_adj_er)

#### Barabási-Albert network :

In [ ]:
type(random_ba)

In [ ]:
degree_ba = np.count_nonzero(random_ba, axis=1)
unique_ba, counts_ba = np.unique(degree_ba, return_counts=True)
counts_ba = counts_ba / float(n_nodes)
weights_ba = dict(zip(unique_ba, counts_ba))

In [ ]:
values_ba = set(range(54, 313))
degrees_ba = set(degree_ba)
print(values_ba - degrees_ba)

In [ ]:
bins_ba = [1]+[55+i for i in range(161)]+[216+7*i for i in range(14)]+[623]

In [ ]:
dist_ba = np.histogram(unique_ba, weights=counts_ba, bins = bins_ba)[0]
dist_adj_ba = np.histogram(unique_adj, weights=counts_adj, bins = bins_ba)[0]

In [ ]:
kl_adj_ba = kl_divergence(dist_ba, dist_adj_ba)
print(kl_adj_ba)

### Question 8

Plot the degree distribution historgrams for all three networks. Are they consistent with the KL divergence results? Explain.

In [ ]:
plt.hist(unique_adj, weights=counts_adj)
plt.title('Degree distribution of our network')

In [ ]:
plt.hist(unique_er, weights=counts_er)
plt.title('Degree distribution of the Erdos network')

In [ ]:
plt.hist(unique_ba, weights=counts_ba)
plt.title('Degree distribution of the Barabasi network')

**Your answer here.**

### Question 9

Imagine you got equal degree distributions. Would that guarantee you got the same graph? Explain.

**Your answer here.**

## Part 2

**You are allowed to use any additional library here (e.g., NetworkX, PyGSP, etc.).** Be careful not to include something here and use it in part 1!

In [ ]:
import networkx as nx

### Question 10

Choose a random network model that fits you network well. Explain your choice. 

**Hint:** Check lecture notes for different network models and their properties. Your choice should be made based on at least one property you'd expect to be similar.

**Your answer here.**

### Question 11

Explain (in short) how the chosen model works.

**Your answer here.**

### Question 12

Create a random graph from that model, such that the number of nodes is the same as in your graph.

In [ ]:
# Watts random graph
G_watts = nx.watts_strogatz_graph(n_nodes, int(k_real), prob)
expected_watts_edges = G_watts.number_of_edges()
print("Watts edges: ", expected_watts_edges)
nx.draw(G_watts)
# Random Dense Graph
G_dense = nx.dense
expected_dense_edges = G_watts.number_of_edges()
print("Watts edges: ", expected_watts_edges)
nx.draw(G_watts)

In [ ]:
# Calculating real k - average degree
k_real = 2 * n_edges / n_nodes
print("Average degree (k) for Original Graph: ",k_real)
# Calculating k - average degree for Random Dense Graph
k_dense = 2 * expected_dense_edges / n_nodes
print("Average degree (k) for Random Dense Graph: ",k_dense)
# Error Calculation
error_dense = (abs(k_real - k_dense)/k_real)*100
print("Error for Random Dense Graph: ", error_dense, "%")

### Question 13

Check the properties you expected to be similar, and compare to your network.

In [ ]:
# Your code here.

Are the results what you expected? Explain.

**Your answer here.**